In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re


# URL to scrapp
BASE_URL = "https://www.autosphere.fr"
# All electric
search_url_template = "https://www.autosphere.fr/recherche?fuel_type=Electrique&from={}"
# Pour un modèle en particulier

# la pagination d'autosphère se fait via un from et avance de 23 en 23 pour le moment
all_links = set()
START_OFFSET = 0 # -> start
STOP_OFFSET = 10
STEP = 23
infos = {} # Pour le df de fin a exporter dans l'excel 

SEARCH_URL_TEMPLATE = "https://www.autosphere.fr/recherche?fuel_type=Electrique&brand=Peugeot&model=2008&from={}"
## On récupère tous les liens en parcourant toutes les pages

all_links = set()
offset = START_OFFSET

while True:
    url = SEARCH_URL_TEMPLATE.format(START_OFFSET)
    print(url)
    print(f"Scraping page avec from={START_OFFSET} ...")
    response = requests.get(url)
    print(response)
    if START_OFFSET > STOP_OFFSET:
        break
    if response.status_code != 200:
        print(f"Page avec from={START_OFFSET} inaccessible. Fin du scraping.")
        break

    soup = BeautifulSoup(response.text, 'html.parser')
    print(soup)
    new_links = set()
    for a_tag in soup.select('a[href*="/fiche"]'):
        print('ici')
        href = a_tag.get('href')
        if href and "/auto-occasion" in href:
            full_url = BASE_URL + href
            new_links.add(full_url)

    if not new_links:
        print(f"Aucun nouveau lien trouvé avec from={START_OFFSET}. Fin du scraping.")
        break

    initial_count = len(all_links)
    all_links.update(new_links)
    added_count = len(all_links) - initial_count

    print(f"{added_count} nouveaux liens trouvés avec from={START_OFFSET}.")
    offset += STEP
    time.sleep(1)

print(f"\nNombre total de fiches uniques récupérées : {len(all_links)}")

## Récupération des infos pour chaque véhicule
car_nbr = 1
for link in all_links:
    infos = {}

    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")  # Décommente pour exécuter sans interface
    driver = webdriver.Chrome(options=options)

    try:
        driver.get(link)
        wait = WebDriverWait(driver, 15)
        all_li = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//li")))

        score_sante = None
        kilometrage = None
        annee = None

        for li in all_li:
            text = li.text.strip()
            if "Santé" in text and not score_sante:
                match = re.search(r"(\d+\s?%)", text)
                if match:
                    score_sante = match.group(1)

            if "km" in text and not kilometrage:
                match = re.search(r"([\d\s]+km)", text)
                if match:
                    kilometrage = match.group(1).strip().replace("\u202f", "").replace(" ", "").replace("km", "").strip()

            if re.search(r"\b20\d{2}\b", text) and not annee:
                annee = re.search(r"\b(20\d{2})\b", text).group(1)

        titre_element = wait.until(EC.presence_of_element_located((By.XPATH, "//h1")))
        titre_text = titre_element.text.strip()

        match = re.search(r"\d+ch\s+\w+", titre_text)

        marque = None
        modele = None
        version_complete = None

        try:
            links_breadcrumb = wait.until(EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "li.inline-flex.items-center a.text-blue-700")
            ))
            if len(links_breadcrumb) >= 2:
                marque = links_breadcrumb[2].text.strip()
                modele = links_breadcrumb[3].text.strip()
            version_span = driver.find_element(By.CSS_SELECTOR, "span.text-black-600")
            version_complete = version_span.text.strip()
            if ' - ' in version_complete:
                version_complete = version_complete[:version_complete.find(' - ')]
            if modele != "2008":
                version_complete = re.sub(r"\b20\d{2}\b", "", version_complete)
            version_complete = version_complete.replace('Achat Integral', "").strip()
        except Exception as e:
            print(f"[{car_nbr}] Erreur fil d’Ariane : {e}")
        print(annee, kilometrage)
        infos["lien"] = link
        infos["SoH"] = score_sante
        infos["Année"] = int(annee)
        infos["Odomètre (km)"] = int(kilometrage)
        infos["Type"] = version_complete
        infos["Modèle"] = modele
        infos["OEM"] = marque

    finally:
        driver.quit()
    # infos[car_nbr] = {}
    # options = webdriver.ChromeOptions()
    # # options.add_argument("--headless")  # Active cette ligne si tu veux en mode sans fenêtre
    # driver = webdriver.Chrome(options=options)

    # driver.get(link)
    # try:
    #     wait = WebDriverWait(driver, 15)

    #     all_li = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//li")))

    #     score_sante = None
    #     kilometrage = None
    #     annee = None

    #     for li in all_li:
    #         text = li.text.strip()

    #         if "Santé" in text and not score_sante:
    #             match = re.search(r"(\d+\s?%)", text)
    #             if match:
    #                 score_sante = match.group(1)

    #         if "km" in text and not kilometrage:
    #             match = re.search(r"([\d\s]+km)", text)
    #             if match:
    #                 kilometrage = match.group(1).strip().replace("\u202f", "").replace(" ", "").replace("km", "").strip()

    #         if re.search(r"\b20\d{2}\b", text) and not annee:
    #             annee = re.search(r"\b(20\d{2})\b", text).group(1)

    #     # Titre principal de la fiche (h1)
    #     titre_element = wait.until(EC.presence_of_element_located((By.XPATH, "//h1")))
    #     titre_text = titre_element.text.strip()

    #     # Version courte si détectée dans le titre
    #     version = None
    #     match = re.search(r"\d+ch\s+\w+", titre_text)
    #     if match:
    #         version = match.group(0)

    #     # === Récupération de la marque, modèle, version complète via fil d’Ariane ===
    #     marque = None
    #     modele = None
    #     version_complete = None

    #     try:
    #         links_breadcrumb = wait.until(EC.presence_of_all_elements_located(
    #             (By.CSS_SELECTOR, "li.inline-flex.items-center a.text-blue-700")
    #         ))
    #         if len(links_breadcrumb) >= 2:
    #             marque = links_breadcrumb[2].text.strip()
    #             modele = links_breadcrumb[3].text.strip()
    #         version_span = driver.find_element(By.CSS_SELECTOR, "span.text-black-600")
    #         version_complete = version_span.text.strip()
    #         if version_complete.find(' - ') > 0:
    #             version_complete = version_complete[:version_complete.find('-')]
    #         if modele !="2008":
    #             version_complete = re.sub(r"\b20\d{2}\b", "",  version_complete)
    #         version_complete = version_complete.strip()

    #     except Exception as e:
    #         print("Erreur lors de la récupération marque/modèle/version:", e)

    #     # Enregistrement pour l'excel 
        
    #     infos[car_nbr]["lien"] = link
    #     infos[car_nbr]['SoH'] = score_sante
    #     infos[car_nbr]['Année'] = annee
    #     infos[car_nbr]['Odomètre (km)'] = kilometrage
    #     infos[car_nbr]['Type'] = version_complete
    #     infos[car_nbr]['Modèle'] = modele
    #     infos[car_nbr]['OEM'] = marque

    #     car_nbr +=1
    # finally:
    #     driver.quit()


In [ ]:
infos

In [ ]:
import pandas as pd
pd.DataFrame(infos).T[["OEM","Modèle","Type","Année","Odomètre (km)","SoH"]].dropna(subset='SoH')

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup


# URL de la fiche
url = "https://www.autosphere.fr/fiche-mixte/auto-occasion-citroen-c4-moteur-electrique-136ch-100-kw-feel-pack-automatique-16430-champniers-214721"

# Configurer Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Ne pas ouvrir une fenêtre
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")  # Décommente pour exécuter sans interface
driver = webdriver.Chrome(options=options)

driver.get(url)

# Récupération du HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Fermer le navigateur
driver.quit()

# Extraire le score de santé (vous pouvez ajuster cette sélection selon le HTML exact)
score = None
for li in soup.find_all("li"):
    if "Santé" in li.text:
        match = re.search(r"score d'intégrité\s+(\d+)\s*%", li.text)
        if match:
            sante_batterie = int(match.group(1))
            score = sante_batterie
        else:
            print("Score d'intégrité non trouvé.")


In [ ]:
score